In [14]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import Input, Embedding, LSTM, Merge, Dense,Concatenate,merge, multiply
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras
import pickle
from keras.models import model_from_json
from keras.models import Model,Sequential
from keras.layers import Input, Embedding, LSTM, Merge, Dense,Concatenate,merge, multiply
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras
import pickle
from keras.models import model_from_json
from keras.models import Model,Sequential
from keras.layers import Input, Embedding, LSTM, Merge,merge,Dense,Dropout
from keras.optimizers import Adadelta,Adam



In [3]:
import tensorflow as tf
from tensorflow import Graph

In [4]:
malstm = "./malstm/malstm.json"
malstm_weights = "./malstm/malstm_weights.h5"
vocab = "./sick_vocab.pickle"
contra = "./final/contra.json"
contra_weights = "./final/contra_weights.h5"

In [5]:
# load json and create model
json_file = open('./malstm/malstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")

print("Loaded model from disk")

OSError: Unable to open file (unable to open file: name = 'model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [6]:
def load_model(model_json, model_weights) : 
    json_file = open(model_json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(model_weights)
    
    return loaded_model


In [7]:
malstm = load_model(malstm, malstm_weights)

In [124]:
class MALSTM : 
    
    def __init__(self, model_json, model_weights, vocab) :
        
        sess = tf.Session()
        K.set_session(sess)
        json_file = open(model_json, 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        self.model = model_from_json(loaded_model_json)
        self.model.load_weights(model_weights)
#         self.model.summary()
        self.vocab = pickle.load(open(vocab, "rb"))

    def output(self, A):
        
        A = self.preprocess(A)                
        output = self.model.predict(A)
        return output
    
    def output_all(self, text) : 
        
        result = []
        for i in text :  
            result.append(self.output(i))
            
        return result
        
        
    def preprocess(self, text):
        word_list = self.text_to_word_list(text)
        ind = []
        for word in word_list : 
            if(word in self.vocab) : 
                ind.append(self.vocab[word])
        ind = pad_sequences([ind], maxlen=26)
        return ind 
    
    def clear(self) : 
        K.set_session(None)
        del self.model
        
    def text_to_word_list(self, text):
        ''' Pre process and convert texts to a list of words '''
        text = str(text)
        text = text.lower()

        # Clean the text
        text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "cannot ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r",", " ", text)
        text = re.sub(r"\.", " ", text)
        text = re.sub(r"!", " ! ", text)
        text = re.sub(r"\/", " ", text)
        text = re.sub(r"\^", " ^ ", text)
        text = re.sub(r"\+", " + ", text)
        text = re.sub(r"\-", " - ", text)
        text = re.sub(r"\=", " = ", text)
        text = re.sub(r"'", " ", text)
        text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
        text = re.sub(r":", " : ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r" u s ", " american ", text)
        text = re.sub(r"\0s", "0", text)
        text = re.sub(r" 9 11 ", "911", text)
        text = re.sub(r"e - mail", "email", text)
        text = re.sub(r"j k", "jk", text)
        text = re.sub(r"\s{2,}", " ", text)

        text = text.split()

        return text

In [18]:
vocab

'./sick_vocab.pickle'

In [72]:
a = "Two young women are sparring in a kickboxing fight"
b = "Two women are sparring in a kickboxing match"

In [81]:
obj = MALSTM(malstm, malstm_weights, vocab)

In [10]:
obj.clear()

In [83]:
vocab1 = pickle.load(open(vocab, "rb"))

In [84]:
vocab1["kids"]

2

In [74]:
A = obj.output(a)
B = obj.output(b)

In [75]:
A

array([[  2.17306279e-02,  -5.01913950e-04,  -2.13505444e-03,
         -1.82000011e-01,  -2.53584921e-01,   2.56872356e-01,
          3.09889406e-01,  -1.57066435e-03,  -8.15546140e-04,
         -1.57062322e-01,   2.41772115e-01,   4.54705775e-01,
          2.78473890e-04,   3.14909779e-02,  -1.07448578e-01,
          6.11151084e-02,  -5.07712141e-02,  -7.55723938e-02,
          3.16619486e-01,   1.73975408e-01,  -4.20948528e-02,
          4.17387782e-04,   1.59952848e-03,  -2.66087125e-03,
          1.58760920e-01,  -3.39385390e-01,   1.30262729e-02,
          1.10630819e-03,   9.83540118e-02,  -3.54880537e-03,
          4.95779179e-02,   9.52103408e-04,  -1.29853627e-02,
         -3.74250680e-01,   3.05484772e-01,  -1.66012838e-01,
         -1.53492361e-01,  -1.51359558e-01,  -3.96886952e-02,
          2.98641175e-02,   1.66826753e-03,   0.00000000e+00,
         -1.39712077e-02,   3.08704138e-01,  -9.08505023e-02,
          1.00915588e-03,  -1.78259483e-03,   5.66147029e-01,
        

In [133]:
class Entailment : 
    
    def __init__(self, ent_json, ent_weights) : 
        
        model2 = Sequential()
        model2.add(Dense(200, input_shape=(100,),kernel_initializer='he_normal', activation='relu'))
        model2.add(Dropout(0.2))
        model2.add(Dense(100, kernel_initializer='he_normal', activation='relu'))
        model2.add(Dropout(0.2))
        model2.add(Dense(50, kernel_initializer='he_normal', activation='relu'))
        model2.add(Dropout(0.2))
        model2.add(Dense(3, activation='softmax'))
        model2.compile(Adam(lr = 0.001),'categorical_crossentropy', metrics=['accuracy'])
        
        self.model = model2
        self.model.load_weights(ent_weights)
        
    def clear(self) : 
        K.set_session(None)   
        del self.model
    
    def predict(self, A, B) : 
        left_matrix = A
        right_matrix = B
        diff_features= np.abs(np.subtract(left_matrix, right_matrix))
        prod = np.multiply(left_matrix,right_matrix)
        extracted_features = np.concatenate([diff_features,prod], axis = 1)
        output = self.model.predict(extracted_features)
        return np.argmax(output)
    
    def predict_all(self, listA, listB) : 
        
        result = []
        for i in range(len(listA)) : 
            result.append(self.predict(listA[i], listB[i]))
        
        return result

In [16]:
import keras.backend as K

In [116]:
del obj.model
K.clear_session()

AttributeError: model

In [82]:
e = Entailment(contra, contra_weights)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 200)               20200     
_________________________________________________________________
dropout_31 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_32 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_33 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 3)                 153       
Total para

In [78]:
e.predict(A, B)

2

In [29]:
K.set_session(a)

In [16]:
a

In [17]:
K.get_session()

In [63]:
K.clear_session()

In [30]:
K.get_session()

In [31]:
K.get_session()

In [32]:
K.set_session(None)

In [34]:
K.get_session()

In [35]:
a

In [41]:
K.set_session(a)

In [42]:
K.get_session()

In [12]:
K.set_session(None)

In [44]:
K.get_session()

In [91]:
x = tf.Session()

In [92]:
x

In [93]:
K.set_session(x)

In [100]:
K.get_session()

In [4]:
sess = tf.Session()

In [5]:
g = Graph()

In [107]:
test = pd.read_csv('./testing_data-2.csv')


In [109]:
test

,Unnamed: 0,Unnamed: 1,pair_ID,sentence_A,sentence_B,entailment_label,relatedness_score,entailment_AB,entailment_BA,sentence_A_original,sentence_B_original,sentence_A_dataset,sentence_B_dataset,SemEval_set
0,0,10.0,10,A brown dog is attacking another animal in fro...,A brown dog is attacking another animal in fro...,ENTAILMENT,4.9,A_entails_B,B_neutral_A,A brown dog is jumping up at another dog in fr...,A brown dog is jumping up at another dog in fr...,FLICKR,FLICKR,TEST
1,1,19.0,19,A person in a black jacket is doing tricks on ...,A man in a black jacket is doing tricks on a m...,ENTAILMENT,4.9,A_entails_B,B_entails_A,A person in a black jacket doing tricks on a m...,A person in a black jacket doing tricks on a m...,FLICKR,FLICKR,TEST
2,2,22.0,22,A skilled person is riding a bicycle on one wheel,A person is riding the bicycle on one wheel,ENTAILMENT,4.3,A_entails_B,B_neutral_A,A person rides their bicycle onto a rock and b...,A person rides their bicycle onto a rock and b...,FLICKR,FLICKR,TEST
3,3,32.0,32,The game of basketball consists of a ball bein...,A man with a jersey is dunking the ball at a b...,ENTAILMENT,4.2,A_entails_B,B_entails_A,A man with jersey 23 is dunking the ball at a ...,A man with jersey 23 is dunking the ball at a ...,FLICKR,FLICKR,TEST
4,4,33.0,33,A man who is playing dunks the basketball into...,The player is dunking the basketball into the ...,ENTAILMENT,4.7,A_entails_B,B_entails_A,Player number 23 dunks the basketball into the...,Player number 23 dunks the basketball into the...,FLICKR,FLICKR,TEST
5,5,41.0,41,Two people are kickboxing and spectators are w...,Two people are fighting and spectators are wat...,ENTAILMENT,4.4,A_entails_B,B_neutral_A,"Two people kick boxing, with others spectating.","Two people kick boxing, with others spectating.",FLICKR,FLICKR,TEST
6,6,52.0,52,Kids in red shirts are playing in the leaves,Children in red shirts are playing in the leaves,ENTAILMENT,4.9,A_entails_B,B_entails_A,Children in red shirts play in the leaves.,Children in red shirts play in the leaves.,FLICKR,FLICKR,TEST
7,7,63.0,63,Two children are lying in the snow and are dra...,Two children are lying in the snow and are mak...,ENTAILMENT,4.4,A_entails_B,B_neutral_A,Two children lying in the snow making snow angels,Two children lying in the snow making snow angels,FLICKR,FLICKR,TEST
8,8,66.0,66,Two people wearing snowsuits are on the ground...,Two people in snowsuits are lying in the snow ...,ENTAILMENT,4.8,A_entails_B,B_entails_A,Two people in snowsuits laying in the snow mak...,Two people in snowsuits laying in the snow mak...,FLICKR,FLICKR,TEST
9,9,74.0,74,A little girl is looking at a woman in costume,A young girl is looking at a woman in costume,ENTAILMENT,4.7,A_entails_B,B_entails_A,The back of a woman in costume ; little girl l...,The back of a woman in costume ; little girl l...,FLICKR,FLICKR,TEST


In [121]:
A = test["sentence_A"]
B = test["sentence_B"]

In [125]:
obj = MALSTM(malstm, malstm_weights, vocab)

In [134]:
e = Entailment(contra, contra_weights)

In [97]:
a = obj.output(A[0])
b = obj.output(B[0])

In [139]:
result_A = obj.output_all(A)

In [140]:
result_B = obj.output_all(B)

In [141]:
res = e.predict_all(result_A, result_B)

In [142]:
res

[1, 1, 1, 2, 2, 0, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1]

In [100]:
A[0]

'A brown dog is attacking another animal in front of the tall man in pants'